In [1]:
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Model, Sequential

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_path = 'data/cifar-10/'
train_path = data_path + 'trainLabels.csv'

# dfs contain filenames under "id" 
# we need to indicate we want our to load this in as string data
train_df = pd.read_csv(train_path, dtype='str')

In [3]:
train_df['label'].value_counts()

cat           5000
deer          5000
airplane      5000
horse         5000
dog           5000
bird          5000
frog          5000
ship          5000
automobile    5000
truck         5000
Name: label, dtype: int64

In [4]:
def append_ext(filename):
    '''
    appends 'png' file extension to a filename (str)
    '''
    return filename+'.png'

In [5]:
train_df['id'] = train_df['id'].apply(append_ext)

In [6]:
train_df.head()

,id,label
0,1.png,frog
1,2.png,truck
2,3.png,truck
3,4.png,deer
4,5.png,automobile


In [7]:
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)

train_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=(data_path+"train/"),
                                            x_col="id",
                                            y_col="label",
                                            subset="training",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(dataframe=train_df,
                                            directory=(data_path+"train/"),
                                            x_col="id",
                                            y_col="label",
                                            subset="validation",
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="categorical",
                                            target_size=(32,32))

Found 40000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


In [9]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.compile(optimizers.RMSprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
1250/1250 [==============================] - 115s 92ms/step - loss: 1.8723 - accuracy: 0.3138 - val_loss: 1.5717 - val_accuracy: 0.4324
Epoch 2/20
1250/1250 [==============================] - 146s 117ms/step - loss: 1.5323 - accuracy: 0.4409 - val_loss: 1.3693 - val_accuracy: 0.5066
Epoch 3/20
1250/1250 [==============================] - 139s 111ms/step - loss: 1.4042 - accuracy: 0.4944 - val_loss: 1.3726 - val_accuracy: 0.5133
Epoch 4/20
1250/1250 [==============================] - 144s 115ms/step - loss: 1.3125 - accuracy: 0.5324 - val_loss: 1.1718 - val_accuracy: 0.5812
Epoch 5/20
1250/1250 [==============================] - 154s 123ms/step - loss: 1.2363 - accuracy: 0.5642 - val_loss: 1.1291 - val_accuracy: 0.6026
Epoch 6/20
1250/1250 [==============================] - 168s 135ms/step - loss: 1.1725 - accuracy: 0.5846 - val_loss: 1.0645 - val_accuracy: 0.6234
Epoch 7/20
1250/1250 [===============